In [6]:
#get aggreate taxi and ride sharding trips from 2015
from pyspark.sql.functions import col
from pyspark.sql import Row
aggregate_all_types_file = "gs://hongzhi-columbia-eecs6893/final/data/aggregate/aggregate_all_types.csv"
tcl_df = spark.read.load(aggregate_all_types_file,format="csv", inferSchema="true", header="true")
def getTaxiAndRiderSharingFrom2015(row):
    validTypes = set(["FHV - High Volume","Green","Yellow"])
    return int(row["Month/Year"].split("-")[0]) >= 2017 and row["License Class"] in validTypes
tcl_df = tcl_df.rdd.filter(getTaxiAndRiderSharingFrom2015).toDF()
#tcl_df.show()
tcl_df = tcl_df.select(tcl_df["Month/Year"].alias("time"),tcl_df["License Class"].alias("type"),\
                       tcl_df["Trips Per Day"].alias("trips"),\
                       tcl_df["Vehicles Per Day"].alias("vehicles"))
               
def updateType(row):
    t = row["type"]
    t = "FHV" if t == "FHV - High Volume" else "Taxi"
    trips = int(row["trips"].replace(',', ''))
    vehicles = int(row["vehicles"].replace(',', ''))
    row["type"] = t
    row["trips"] = trips
    row["vehicles"] = vehicles
    return row
tcl_df = tcl_df.rdd.map(lambda row: row.asDict(True)).map(updateType).toDF()
#tcl_df.show()
tcl_agg = tcl_df.groupby("time","type").agg({"trips":"sum", "vehicles":"sum"}).orderBy("time","type")\
      .withColumnRenamed("sum(vehicles)", "vehicles").withColumnRenamed("sum(trips)","trips")
tcl_agg.printSchema()


root
 |-- time: string (nullable = true)
 |-- type: string (nullable = true)
 |-- trips: long (nullable = true)
 |-- vehicles: long (nullable = true)



In [16]:
#get aggregate citi bike trips from 2017
citibike_trip_files = "gs://hongzhi-columbia-eecs6893/final/data/citibike/*.csv"
citi_df_orig_rest = spark.read.load(citibike_trip_files,format="csv", inferSchema="true", header="true")
citi_df_orig_rest.printSchema()

root
 |-- tripduration: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- stoptime: timestamp (nullable = true)
 |-- start station id: string (nullable = true)
 |-- start station name: string (nullable = true)
 |-- start station latitude: double (nullable = true)
 |-- start station longitude: double (nullable = true)
 |-- end station id: string (nullable = true)
 |-- end station name: string (nullable = true)
 |-- end station latitude: double (nullable = true)
 |-- end station longitude: double (nullable = true)
 |-- bikeid: integer (nullable = true)
 |-- usertype: string (nullable = true)
 |-- birth year: string (nullable = true)
 |-- gender: integer (nullable = true)



In [17]:
#citibike_trip_2017_files = "gs://hongzhi-columbia-eecs6893/final/data/citibike/2017first3/*.csv"
#citi_df_orig_2017 = spark.read.load(citibike_trip_2017_files,format="csv", inferSchema="true", header="true")
#citi_df_orig_2017.printSchema()

root
 |-- Trip Duration: integer (nullable = true)
 |-- Start Time: timestamp (nullable = true)
 |-- Stop Time: timestamp (nullable = true)
 |-- Start Station ID: integer (nullable = true)
 |-- Start Station Name: string (nullable = true)
 |-- Start Station Latitude: double (nullable = true)
 |-- Start Station Longitude: double (nullable = true)
 |-- End Station ID: integer (nullable = true)
 |-- End Station Name: string (nullable = true)
 |-- End Station Latitude: double (nullable = true)
 |-- End Station Longitude: double (nullable = true)
 |-- Bike ID: integer (nullable = true)
 |-- User Type: string (nullable = true)
 |-- Birth Year: integer (nullable = true)
 |-- Gender: integer (nullable = true)



In [24]:
citi_df = citi_df_orig_rest.select(citi_df_orig_rest["starttime"].alias("time"), "bikeid")
def updateRow(row):
    t = row["time"]
    month = str(t.month) if t.month >= 10 else "0"+str(t.month)
    t = [str(t.year), month, str(t.day)]
    row["time"] = "-".join(t)
    row["type"] = "Citibike"
    row["count"]= 1
    return row
citi_df_updated = citi_df.rdd.map(lambda row: row.asDict(True)).map(updateRow).toDF()

citi_df_agg = citi_df_updated.groupby("time","type","bikeid").agg({"count":"sum"})\
                             .withColumnRenamed("sum(count)","trips").orderBy("time","type","bikeid")
citi_df_agg = citi_df_agg.groupby("time","type").agg({"bikeid":"count", "trips":"sum"})\
                             .withColumnRenamed("sum(trips)","trips").withColumnRenamed("count(bikeid)","vehicles")
def updatetime(row):
    row["time"] = "-".join(row["time"].split('-')[:2])
    return row
citi_df_agg_by_month = citi_df_agg.rdd.map(lambda row: row.asDict(True)).map(updatetime).toDF()
citi_df_avg_by_month = citi_df_agg_by_month.groupby("time","type").agg({"vehicles":"avg", "trips":"avg"})\
                                           .withColumnRenamed("avg(vehicles)","vehicles")\
                                           .withColumnRenamed("avg(trips)","trips")
citi_df_avg_by_month.printSchema()


root
 |-- time: string (nullable = true)
 |-- type: string (nullable = true)
 |-- trips: double (nullable = true)
 |-- vehicles: double (nullable = true)



In [25]:
citi_df_avg_by_month_int = citi_df_avg_by_month.withColumn("trips",citi_df_avg_by_month["trips"].cast("int"))\
                                               .withColumn("vehicles",citi_df_avg_by_month["vehicles"].cast("int"))
citi_df_avg_by_month_int.show()

+-------+--------+-----+--------+
|   time|    type|trips|vehicles|
+-------+--------+-----+--------+
|2017-12|Citibike|28708|    6268|
|2019-04|Citibike|58869|    9507|
|2018-05|Citibike|58861|    8839|
|2017-09|Citibike|62603|    8445|
|2018-04|Citibike|43584|    8063|
|2019-07|Citibike|70356|   10481|
|2019-05|Citibike|62082|    9125|
|2018-06|Citibike|65103|    8681|
|2017-04|Citibike|43846|    7161|
|2017-10|Citibike|61212|    8052|
|2017-11|Citibike|44354|    8362|
|2017-08|Citibike|58596|    7946|
|2017-06|Citibike|57719|    7238|
|2019-01|Citibike|31202|    7426|
|2017-05|Citibike|49137|    7260|
|2018-09|Citibike|62596|    7504|
|2019-09|Citibike|81496|   11548|
|2019-08|Citibike|75620|   10482|
|2018-02|Citibike|30111|    5971|
|2019-06|Citibike|70845|    9846|
+-------+--------+-----+--------+
only showing top 20 rows



In [26]:
total_df = tcl_agg.union(citi_df_avg_by_month_int).orderBy("time","type")
total_df.show()

+-------+--------+------+--------+
|   time|    type| trips|vehicles|
+-------+--------+------+--------+
|2017-01|     FHV|344972|   32782|
|2017-01|    Taxi|347726|   15908|
|2017-02|     FHV|375693|   34850|
|2017-02|    Taxi|363960|   16007|
|2017-03|     FHV|402938|   35619|
|2017-03|    Taxi|369416|   15935|
|2017-04|Citibike| 43846|    7161|
|2017-04|     FHV|393918|   36750|
|2017-04|    Taxi|370889|   16032|
|2017-05|Citibike| 49137|    7260|
|2017-05|     FHV|399546|   38015|
|2017-05|    Taxi|360031|   15899|
|2017-06|Citibike| 57719|    7238|
|2017-06|     FHV|420017|   40408|
|2017-06|    Taxi|354421|   15667|
|2017-07|Citibike| 55987|    7516|
|2017-07|     FHV|421285|   40679|
|2017-07|    Taxi|306545|   15046|
|2017-08|Citibike| 58596|    7946|
|2017-08|     FHV|429691|   41372|
+-------+--------+------+--------+
only showing top 20 rows



In [27]:
#save to storage
total_df_path = "gs://hongzhi-columbia-eecs6893/final/analysis/all_types_aggregate.csv"
total_df.write.save(total_df_path,format="csv", header="true", mode="overwrite")

In [28]:
#load into big query
import subprocess
files = total_df_path + '/part-*'
subprocess.check_call(
        'bq load --source_format=CSV '
        '--replace '
        '--autodetect '
        'final.all_types_aggregate {files}'.format(
        files=files
        ).split())

0